Large language models (LLMs) can perform all kinds of tasks ranging from translation over summarization to text generation or even image and video generation.
Usually, there are LLMs readily available for any kind of task, which can be easily found on [HuggingFace](https://huggingface.co/models).
However, if there is no available model doing just what you want, then you use a pre-trained foundation or base model and fine-tune it for the specific task it should perform.

In my case, I was looking for a model to answer questions about long documents in natural language.
Most models I could find were limited to short context lengths, i.e., could not handle entire documents as input, or were not trained to output generated natural answers (see my post [here](https://stefanbschneider.github.io/blog/posts/generative-qa/)). 

In this blog post, I fine-tune a pre-trained Longformer Encoder-Decoder (LED) model for generative question answering.
Fine-tuning builds on available foundation or base models, i.e., LLMs that were pre-trained on massive amounts of data (usually Internet scale) in a self-supervised fashion.
During fine-tuning, such a base model is then further trained in a supervised way on a much smaller task-specific dataset.
As a result, fine-tuning is much faster and cheaper than pre-training from scratch, but it does require a suitable dataset.



# Base Model

As base model, I use the Longformer Encoder-Decoder (LED) from AllenAI: [allenai/led-base-16384](https://huggingface.co/allenai/led-base-16384)

There is a fine-tuned version of this model for [text summarization](https://huggingface.co/allenai/led-large-16384-arxiv), but I did not see any for question answering.

TODO: Demonstrate lack of question answering when using the base model




# Task-Specific Dataset (Long-Form Question Answering)

My task of interest is answering questions in natural language given a (potentially long) context.
Since I do not have the means of collecting and creating my own dataset, I was looking for a suitable dataset online.

The well-known [SQuAD dataset](https://huggingface.co/datasets/rajpurkar/squad_v2) is only suitable for extractive question answering, where the answer is a span text from the provided context.
The [DuoRC dataset](https://huggingface.co/datasets/ibm-research/duorc) with questions and answers about a given movie plot can be used for both extractive and generative/abstracitve Q&A.
However, I found the answers to be overly short, often just a few words, and not always very natural.

Finally, I found a [suitable dataset for long-form question answering (LFQA)](https://huggingface.co/datasets/LLukas22/lfqa_preprocessed) with natural, multi-sentence answers to questions based on provided contexts.
Other, similar datasets are the [ELI5 dataset](https://huggingface.co/datasets/defunct-datasets/eli5) (no longer available) and the [natural questions dataset](https://huggingface.co/datasets/google-research-datasets/natural_questions).

Looking at the long-tail distribution of answer lengths in my selected dataset, I found that a few answers were overly long (up to ~6000 tokens).

![Distribution of answer lengths in the original dataset (in number of tokens).](images/answer-lengths-tokens-original.png)

Since I did not want my fine-tuned model to create overly long answers, I filtered these examples out of the dataset and made my own version of the dataset with answers only up 512 tokens.
This means the maximum answer length is roughly 12x shorter at the cost of 10% less training data.

My filtered dataset is available here: [stefanbschneider/lfqa-max-answer-length-512]

An example in the dataset looks like this:

```json
{
    "question": "what's the difference between a forest and a wood?",
    "answer": "They're used interchangeably a lot. You'll get different answers from different resources, but the ...",
    "context": [
        "Wood is divided, according to its botanical origin, into two kinds: softwoods, ...",
        "Processing and products differs especially with regard to the distinction between softwood and hardwood ..."
    ]
}
```



# Fine-Tuning

Now that the dataset is ready, the data has to be prepared and the model has to be loaded and configured for fine-tuning.

First, let's install all dependencies.

In [ ]:
%%capture --no-display
%pip install -U pypdf torch transformers

In [2]:
from transformers import pipeline

extractive_qa = pipeline(task="question-answering")

# Abstract from "Attention is all you need" by Vaswani et al.: https://arxiv.org/abs/1706.03762
abstract = """The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task...
"""
question = "What's a transformer'?"

extractive_qa(question=question, context=abstract)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


{'score': 0.4559027850627899,
 'start': 287,
 'end': 302,
 'answer': 'the Transformer'}